In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import gc
import glob

# 그래프 기본 테마 설정
# https://coldbrown.co.kr/2023/07/%ED%8C%8C%EC%9D%B4%EC%8D%AC-%EC%8B%A4%EC%A0%84%ED%8E%B8-08-seaborn-sns-set%EC%9D%84-%ED%86%B5%ED%95%B4-%EC%8A%A4%ED%83%80%EC%9D%BC-%EC%84%A4%EC%A0%95%ED%95%98%EA%B8%B0/
sns.set()

# 그래프 기본 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
# plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['figure.figsize'] = 12, 6
plt.rcParams['font.size'] = 14
plt.rcParams['axes.unicode_minus'] = False


# 복잡한 통계 처리를 위한 라이브러리
from scipy import stats

In [4]:
data_type = "train"
# month = "07"
# category = "청구입금정보"

# local
root_path = '../data/open'

# colab
# root_path = '/content/drive/MyDrive/12조 파이널프로젝트/data'

drive_folder = f'{root_path}/{data_type}/4.청구입금정보/'

In [ ]:
# csv 파일 데이터를 읽어온다.
billing = pd.read_parquet(f'{drive_folder}train_청구정보_통합.parquet')
billing

In [3]:
# 메모리 사용량이 너무 많아서, int64를 int32로 바꾼다.
for col in billing.select_dtypes(include='int64').columns:
    billing[col] = billing[col].astype('int32')

In [4]:
# 출력 옵션 조정 (생략 없이 dtypes 포함 모든 열 보기)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [5]:
# 🔎 결측치 확인
# print("🔍 [결측치 개수]")
# print("▶ billing:\n", billing.isnull().sum().sort_values(ascending=False))

- 결측치는 한개도 없네요

In [7]:
# 🔎 데이터 타입
print("▶ billing dtypes\n", billing.dtypes)

▶ billing dtypes
 기준년월                 int32
ID                  object
대표결제일                int32
대표결제방법코드            object
대표청구지고객주소구분코드       object
대표청구서수령지구분코드        object
청구서수령방법             object
청구서발송여부_B0           int32
청구서발송여부_R3M          int32
청구서발송여부_R6M          int32
청구금액_B0              int32
청구금액_R3M             int32
청구금액_R6M             int32
포인트_마일리지_건별_B0M      int32
포인트_마일리지_건별_R3M      int32
포인트_포인트_건별_B0M       int32
포인트_포인트_건별_R3M       int32
포인트_마일리지_월적립_B0M     int32
포인트_마일리지_월적립_R3M     int32
포인트_포인트_월적립_B0M      int32
포인트_포인트_월적립_R3M      int32
포인트_적립포인트_R12M       int32
포인트_적립포인트_R3M        int32
포인트_이용포인트_R12M       int32
포인트_이용포인트_R3M        int32
포인트_잔여포인트_B0M        int32
마일_적립포인트_R12M        int32
마일_적립포인트_R3M         int32
마일_이용포인트_R12M        int32
마일_이용포인트_R3M         int32
마일_잔여포인트_B0M         int32
할인건수_R3M            object
할인금액_R3M             int32
할인건수_B0M            object
할인금액_B0M             int32
할인금액_청구서_R3M         int32
할인금액_청구서_B

- ID, 대표결제방법코드, 대표청구지고객주소구분코드, 대표청구서수령지구분코드
- 청구서수령방법, 할인건수_R3M , 할인건수_B0M
- 위는 모두 object, 나머지는 int64 --> int32

In [9]:
# 🔎 빈 문자열 확인
# print("\n🔍 [빈 문자열 개수]")
# print("▶ billing 빈 문자열:\n", (billing == '').sum())

- 빈 문자열도 없네요

In [11]:
billing['대표결제방법코드'].unique()

array(['자동이체'], dtype=object)

- 대표결제방법코드 --> 자동이체 뿐
- 날리겠습니다.

In [13]:
billing.drop(columns=['대표결제방법코드'], inplace=True)

In [14]:
# csv너무 느려서 parquet으로 다시 . .
billing.to_parquet(f'{drive_folder}train_청구정보_통합_전처리1.parquet', index=False)

In [6]:
# 잔액정보 (모든 월 통합) 통합 parquet
billing_eda = pd.read_parquet(f'{drive_folder}train_청구정보_통합_전처리1.parquet')
billing_eda

,기준년월,ID,대표결제일,대표청구지고객주소구분코드,대표청구서수령지구분코드,청구서수령방법,청구서발송여부_B0,청구서발송여부_R3M,청구서발송여부_R6M,청구금액_B0,...,할인금액_청구서_B0M,상환개월수_결제일_R6M,상환개월수_결제일_R3M,선결제건수_R6M,선결제건수_R3M,연체건수_R6M,연체건수_R3M,혜택수혜금액_R3M,포인트_마일리지_환산_B0M,혜택수혜금액
0,201807,TRAIN_000000,27,미확인,당사페이앱+이메일,문자메세지,1,1,1,12226,...,0,5,3,0,0,1,0,3,0,0
1,201807,TRAIN_000001,13,주거지,우편,우편,1,1,1,5834,...,163,6,3,0,0,0,0,0,0,0
2,201807,TRAIN_000002,1,미확인,이메일,이메일,1,1,1,21866,...,0,6,3,0,0,0,0,121,0,50
3,201807,TRAIN_000003,5,주거지,우편,우편,1,1,1,16356,...,0,5,3,2,0,0,0,3,0,2
4,201807,TRAIN_000004,13,주거지,우편,우편,0,1,1,0,...,0,1,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2399995,201812,TRAIN_399995,25,주거지,우편,우편,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2399996,201812,TRAIN_399996,20,미확인,이메일,이메일,1,1,1,14402,...,0,6,3,0,0,0,0,164,0,53
2399997,201812,TRAIN_399997,20,회사,우편,우편,1,1,1,5731,...,0,6,3,0,0,0,0,0,0,0
2399998,201812,TRAIN_399998,20,주거지,우편,우편,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
